In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import betterdata
import configparser
import json
import math
from calendar import month_name
# from deep_translator import GoogleTranslator

In [2]:
#Set the dates for the report and the config file with all the paths
initial_date = '2022-01-01'
quarter_start = dt.datetime.strptime(initial_date, '%Y-%m-%d')
end_date = '2022-04-01'
quarter_end = dt.datetime.strptime(end_date, '%Y-%m-%d')
report_date = dt.datetime.today()
today_str = dt.date(report_date.year, report_date.month,report_date.day)
config = configparser.ConfigParser()
config.read('config.ini')
current_dir = config['output'].get('out_dir')
dir_credentials = config['database'].get('cred_dir')
name_file = config['database'].get('cred_file')

In [3]:
connection = betterdata.create_database_connection(betterdata.read_credentials_file_local(dir_credentials = dir_credentials, file_credentials = name_file))


## Queries

In [4]:

with open('querys_redshift/bettercoins.sql', 'r') as file:
    _q_bettercoins = file.read()
    _q_bettercoins = _q_bettercoins.format(date_end_quarter=end_date, date_start_quarter=initial_date)
df_bettercoins = betterdata.get_data_sql(connection=connection, query=_q_bettercoins, close=False)

df_bettercoins['bettercoins_donadas'] = abs(df_bettercoins['bettercoins_donadas'])
df_bettercoins['proporcion_uso_moneda'] = abs(df_bettercoins['proporcion_uso_moneda'])

In [5]:
with open('querys_redshift/bettercoins_habits.sql', 'r') as file:
    _q_bettercoins_habits = file.read()
    _q_bettercoins_habits = _q_bettercoins_habits.format(date_end_quarter=end_date, date_start_quarter=initial_date)
df_bettercoins_habits = betterdata.get_data_sql(connection=connection, query=_q_bettercoins_habits, close=False)

In [6]:
with open('querys_redshift/date_cohort.sql', 'r') as file:
    _q_date_cohort= file.read()
    _q_date_cohort = _q_date_cohort.format(date_end_quarter=end_date, date_start_quarter=initial_date)
df_date_cohorts = betterdata.get_data_sql(connection = connection, query = _q_date_cohort, close = False)
df_cohorts = df_date_cohorts[['organization','n_athletes','date_cohort','diff_dias','total_days']]

In [7]:
with open('querys_redshift/subscribers.sql', 'r') as file:
        _q_subscribers= file.read()
df_subscribers = betterdata.get_data_sql(connection = connection, query = _q_subscribers, close = False)
conditions = [
    (df_subscribers['pk_dim_documenttype'] == 1),
    (df_subscribers['pk_dim_documenttype'] != 1),
    ]
values = [df_subscribers['bk_identification_value'].str[:-1], df_subscribers['bk_identification_value']]
df_subscribers['bk_identification_value'] = np.select(conditions, values)
df_subscribers = df_subscribers.drop(columns = "pk_dim_documenttype")

In [8]:
with open('querys_redshift/benefits.sql', 'r') as file:
    _q_benefits= file.read()
df_benefits = betterdata.get_data_sql(connection = connection, query = _q_benefits, close = False)

In [9]:
with open('querys_redshift/level.sql', 'r') as file:
        _q_level= file.read()
        _q_level= _q_level.format(date_end_quarter=end_date)
df_levels = betterdata.get_data_sql(connection = connection, query = _q_level, close = False)
df_levels = df_levels.fillna('2022-03-31')
df_levels = df_levels.drop_duplicates(subset=['athlete'],keep='last')
df_levels=df_levels.groupby(['name_organization','nivel']).agg({'athlete':'count'}).reset_index()

In [10]:
with open('querys_redshift/dados_baja.sql', 'r') as file:
        _q_level= file.read()
        _q_level= _q_level.format(date_end_quarter=end_date)
df_baja = betterdata.get_data_sql(connection = connection, query = _q_level, close = False)


In [11]:
# out=['REWARDS','PATRICIO TOLEDO','FREE TRIAL','CALLEJAS VARGAS','C.E.COLEGIO ALTAZOR','B2C_BETTERFLY','AGRICOLA LEIVA','LONGPORT CHILE','GUILLERMO RIVEROS F.','C.M.ESTACION CENTRAL','GSC','P&G CHILE','P&G COLOMBIA','P&G PERU','SCHUTZ','SAMSUNG_PROFESORES','SAMSUNG_PROMO', 'AGQ CHILE', 'STARCLEAN Y TUPLAN']
with open('querys_redshift/onboarding.sql', 'r') as file:
    _q_onboarding = file.read()
    _q_onboarding = _q_onboarding.format(date_end_quarter=end_date)
df_onboarding = betterdata.get_data_sql(connection = connection, query = _q_onboarding, close = False)
# df_onboarding = df_onboarding[~df_onboarding.organization.isin(out)]

In [12]:
with open('querys_redshift/date_inicio.sql', 'r') as file:
        _q_date_inicio= file.read()
df_inicio = betterdata.get_data_sql(connection = connection, query = _q_date_inicio, close = False)
df_orgs = pd.merge(df_onboarding, df_inicio, on='organization')
# orgs = list(df_orgs['organization'])
# df_onboarding = df_orgs
# df_onboarding = df_onboarding.drop_duplicates(subset='organization',keep='last')

In [13]:

with open('querys_redshift/companies.sql', 'r') as file:
        _q_date_inicio= file.read()
df_companies = betterdata.get_data_sql(connection = connection, query = _q_date_inicio, close = False)

In [14]:
empresas_q1 = pd.read_excel('Lanzamiento EB Portal.xlsx',sheet_name='admins_file_to_luciano') 
empresas_q1.dropna(subset=['Admin__r.Email'],inplace=True)
empresas_q1.rename(columns={'Unnamed: 0':'id'},inplace=True)
# empresas_q1 = empresas_q1.dropna(subset=['Unnamed: 3'])
# empresas_q1.rename(columns={'Unnamed: 3':'name','Unnamed: 8':'correo','Unnamed: 6':'Nombre','Unnamed: 7':'Apellido'},inplace=True)
# empresas_q1 = empresas_q1[['name','Nombre','correo','Apellido','Unnamed: 9',	'Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17']]
# con_razon = pd.merge(df_onboarding,empresas_q1, how='left',on='id')

In [22]:
ya_estan = df_onboarding[~df_onboarding['id'].isin(empresas_q1['id'])]

In [23]:
no_estan = df_onboarding[df_onboarding['id'].isin(empresas_q1['id'])]

In [24]:
empresas_q4 = pd.read_excel('empresas q4.xlsx') 


In [25]:
ya_estan = pd.merge(ya_estan,empresas_q4,how='left',on='id')

In [26]:
no_estan = pd.merge(no_estan,empresas_q1,how='left',on='id')

In [99]:
ya_estan.to_excel('ya_estan.xlsx')
no_estan.to_excel('no_estan.xlsx')

In [15]:
with open('querys_redshift/clusters.sql', 'r') as file:
        _q_org= file.read()
df_cluster = betterdata.get_data_sql(connection = connection, query = _q_org, close = False)
with open('querys_redshift/organization.sql', 'r') as file:
        _q_org= file.read()
df_org = betterdata.get_data_sql(connection = connection, query = _q_org, close = False)
total = pd.merge(df_cluster, df_org, how='left', on='pk_dim_organization')
total.rename({'cluster': 'labels', 'organization': 'name_organization'}, axis=1, inplace=True)
total = total.drop(columns='athlete')
total['freq'] = total.groupby(['name_organization','labels'])['labels'].transform('count')
df_cluster =  total.drop_duplicates()

In [16]:
with open('querys_redshift/onb_4abril.sql', 'r') as file:
        _q_org= file.read()
df_onb4abril = betterdata.get_data_sql(connection = connection, query = _q_org, close = False)
df_onb4abril['date_inicio'] = pd.to_datetime(df_onb4abril['date_inicio'], format='%Y-%m-%d')
# df_onb4abril[df_onb4abril['date_inicio']== dt.datetime.strptime('2022-04-01','%Y-%m-%d')]

In [17]:
_q_uf = "select value from trusted.dim_ufvalues where \"date\" = '{date_end_quarter}'::date".format(date_end_quarter = '2021-12-31')
uf_value = betterdata.get_data_sql(connection = connection, query = _q_uf, close = False)
uf_value = uf_value.values
uf_value = uf_value[0][0]

In [18]:
with open('querys_redshift/coverage.sql', 'r') as file:
    _q_coverage_f= file.read()
    _q_coverage = _q_coverage_f.format(date_start_quarter=initial_date, date_end_quarter=end_date)
df_coverage = betterdata.get_data_sql(connection = connection, query = _q_coverage, close = False)
df_coverage['coverage'] = df_coverage['coverage_uf']*uf_value

df_coverage_mes=df_coverage.groupby(['organization']).sum()[['coverage']].reset_index()

In [19]:
with open('querys_redshift/coverage_reales.sql', 'r') as file:
    _q_coverage_= file.read()
    _q_coverage_r = _q_coverage_.format(date_start_quarter=initial_date, date_end_quarter=end_date)
_q_coverage_r = betterdata.get_data_sql(connection = connection, query = _q_coverage_r, close = False)
_q_coverage_r['coverage'] = _q_coverage_r['coverage_uf']*uf_value


df_coverage_mes_r=_q_coverage_r.groupby(['organization']).sum()[['coverage']].reset_index()

In [20]:
with open('querys_redshift/activities.sql', 'r') as file:
    _q_activities = file.read()
    _q_activities = _q_activities.format(date_start_quarter=initial_date, date_end_quarter=end_date)
df_activities_by_day_ = betterdata.get_data_sql(connection = connection, query = _q_activities, close = False)
df_steps=df_activities_by_day_[df_activities_by_day_['habit_type']==2]
df_activities_by_day = df_activities_by_day_[df_activities_by_day_['habit_type']==1]

#actividades todo el año
df_activities = df_activities_by_day.fillna(0).groupby(['organization', 'sport'])[['calories', 'minutes_activities']]\
                                        .agg(numCalories = ('calories','sum'), numMinutes= ('minutes_activities','sum'))\
                                        .reset_index()
#steps por mes
df_steps_mes = df_steps.copy()
df_steps_mes['mes'] = df_steps['month_selected'].apply(lambda x: x.strftime('%B'))
df_steps_mes = df_steps_mes.groupby(['organization','mes']).agg({'steps':'sum'}).reset_index()

#activities day por mes
df_activities_by_day['mes'] = df_activities_by_day['month_selected'].apply(lambda x: x.strftime('%B'))
df_activities_by_day = df_activities_by_day[['organization','mes','calories' , 'minutes_activities' ,'sport',  'id']]

#steps todo el año
df_steps = df_steps.groupby('organization').agg({'steps':sum}).reset_index()

C:\Users\ignac\AppData\Local\Temp/ipykernel_13028/3952708930.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_activities_by_day['mes'] = df_activities_by_day['month_selected'].apply(lambda x: x.strftime('%B'))


In [21]:
with open('querys_redshift/meditation.sql', 'r') as file:
    _q_med = file.read()
    _q_med = _q_med.format(date_start_quarter=initial_date, date_end_quarter=end_date)
df_meditation_mes = betterdata.get_data_sql(connection = connection, query = _q_med, close = False)
# #meditation por mes
# df_meditation_mes['month_selected'] = df_meditation_mes['month_selected'].apply(lambda x: x.strftime('%B'))
# df_meditation_mes = df_meditation_mes[['pk_dim_athlete','organization','month_selected','num_meditations','min_meditation']]

# #meditacion total año
df_meditation = df_meditation_mes.groupby(['athlete','organization']).agg({'min_meditation':'sum','num_meditations':'sum'}).reset_index()

In [22]:
with open('querys_redshift/donations.sql', 'r') as file:
    _q_donations_b= file.read()
    _q_donations_b = _q_donations_b.format(date_start_quarter=initial_date, date_end_quarter=end_date)
df_donations_b = betterdata.get_data_sql(connection = connection, query = _q_donations_b, close = False)
df_donations_b['mes'] = df_donations_b['month_selected'].apply(lambda x: x.strftime('%B'))

#donaciones anuales por campaña
df_donations = df_donations_b.groupby(['id_campaign','organization','campaign','type_donation']).agg({'donations':'sum'}).reset_index()

In [23]:
with open('querys_redshift/manual_habits.sql', 'r') as file:
    _q_donations_b= file.read()
    _q_donations_b = _q_donations_b.format(date_start_quarter=initial_date, date_end_quarter=end_date)
df_manual = betterdata.get_data_sql(connection = connection, query = _q_donations_b, close = False)

## Funciones

In [24]:
def get_message(df_coverage, df_meditation, df_activities,organization):
    """
    Defines a message related to an accomplishment of the organization.
    """
    df_med=df_meditation[['organization','min_meditation']].groupby(['organization']).sum()
    df_cal=df_activities[['organization','numCalories']].groupby(['organization']).sum()
    df=df_cal.merge(df_med,on='organization',how='outer')
    df_coverage = df_coverage[['organization','coverage']].groupby(['organization']).sum().reset_index()
    df_final=df_coverage.merge(df,on='organization',how='outer')
    med=df_final.sort_values(by='min_meditation',ascending=False)['organization'].to_numpy()[0:10]
    df_final=df_final.set_index('organization')
    df_c=df_final.drop(med)
    df_c.reset_index(inplace=True)
    cov=df_c.sort_values(by='coverage',ascending=False)['organization'].to_numpy()[0:10]
    df_c=df_c.set_index('organization')
    df_cal=df_c.drop(cov)
    df_cal.reset_index(inplace=True)
    cal=df_cal.sort_values(by='numCalories',ascending=False)['organization'][0:10].to_numpy()
    if organization in med:
        frase= "Están dentro de las 10 empresas que más ha meditado. Mentes en calma, ideas brillantes. "
    elif organization in cov:
        frase="Están dentro de las 10 empresas que más  cobertura de seguro de vida han acumulado. Su protección financiera es tranquilidad para ellos y sus seres queridos. "
    elif organization in cal:
        frase="Están dentro de las 10 empresas que más han quemado calorías. Tu equipo es uno de los más activos y su impacto es el reflejo de su esfuerzo."
    else:
        frase = " ¡Felicitaciones por haberse embarcado en este viaje de transformación e impacto! El bienestar de tus colaboradores y la protección financiera de sus seres queridos es el beneficio más grande que le puedes entregar."
    return frase

In [25]:
aux = df_donations.groupby('organization').agg({'donations':'sum'}).reset_index()
aux[aux['donations']>= 15]

,organization,donations
0,23People,102
1,3G OFFICE,107
2,4Geeks CL SpA,28
3,734 Odontológica P.V,105
5,AB InBev,1438
...,...,...
768,YAPO.CL,245
769,YAPP,98
770,YOM,294
771,YOUTOPIA,132


In [27]:
def dict_summary(df_onboarding, df_activities, df_coverage_mes, df_donations, df_bettercoins, df_levels, organization):
    """
    Creates dictionary with principal values of an organization. (First section of the report)
    """
    df_summary = pd.DataFrame()
    df_summary['numSuscribers'] = df_onboarding.loc[df_onboarding['organization'] == organization, 'num_subscribers'].values
    df_summary['percActivation'] = df_onboarding.loc[df_onboarding['organization'] == organization, 'perc_activation'].values
    df_summary['numSteps'] = df_steps[df_steps['organization']==organization]['steps'].sum()
    df_summary['numCalories'] = df_activities.loc[df_activities['organization'] == organization, 'numCalories'].sum()
    df_summary['bettercoins'] = df_bettercoins[df_bettercoins['organization']==organization]['bettercoins'].values[0]
    df_summary['coverage'] = math.trunc(df_coverage_mes[df_coverage_mes['organization']==organization]['coverage'].sum())
    df_summary['percBettercoins'] = df_bettercoins[df_bettercoins['organization']==organization]['proporcion_uso_moneda'].values[0]
    df_summary['numDonations'] = np.trunc(df_donations.loc[df_donations['organization'] == organization,'donations']).sum()
    c = df_donations[df_donations['organization']==organization]
    
    c=c.groupby('campaign').sum().reset_index()
    causes=c['donations'].to_numpy()
    n_causes=0

    if organization=='SANTANDER':
        n_causes = 1
    elif organization == 'CRUZADOS':
        n_causes = 1
    else:
         for i in causes:
            if i>=1:
                n_causes=n_causes+1
    df_summary['numCauses'] = n_causes
    df_summary['message'] = get_message(df_coverage, df_meditation_mes, df_activities,organization)
    summary = df_summary.T[0].to_dict()
    levels = dict_levels(df_levels, organization)
    summary['level'] = levels['nivel']
    summary['perc_level'] = levels['perc']
    return summary

In [28]:
def get_message_steps(steps: int):
    """
    arreglar porcentajes
    Defines a message related to the distance associated to the tptal amount of steps on the organization.
    """ #cambiar a 1312 pasos por kilometro
    km = steps/1500
    
    if km <= 3000: #santiago a rio
        viajes = math.trunc(km/42)
        porcentaje = math.trunc((km*100)/3000)
        en_pasos = 3000*1312
        if 50 >  porcentaje >  49:
            frase = 'Tu organización ha recorrido medio camino de Santiago a Río de Janeiro. ¡Le faltan ' + f"{math.trunc(3000-km):,}".replace(',','.') + ' kilómetros para llegar!'
        elif 100 >=  porcentaje >=  97:
            frase = '¡Felicidades! Tu organización ha recorrido el camino completo desde Santiago a Río de Janerio. ¡Esos son 3.000 kilómetros!'
        else:
            frase =  'Tu organización ha recorrido el ' + str(porcentaje) + '% ' + 'del camino entre Santiago y Río de Janeiro. ¡Le faltan ' +  f"{math.trunc(3000-km):,}".replace(',','.') + ' kilómetros para llegar!'
    elif  km > 3000 and km <= 21000: #muralla china 21 km
        viajes = math.trunc(km/1406)
        porcentaje = math.trunc((km*100)/21000)
        en_pasos = 21000*1312
        if 50 >  porcentaje >  49:
            frase = 'Tu organización ha recorrido la mitad de la Muralla China. ¡Le faltan ' +  f"{math.trunc(21000-km):,}".replace(',','.') + ' kilómetros para recorrerla completa!'
        elif 100 >=  porcentaje >=  97:
            frase = '¡Felicidades! Tu organización ha recorrido toda la Muralla China. ¡Esos son 21.000 kilómetros!'
        else:
            frase =  'Tu organización ha recorrido el ' + str(porcentaje) + '% ' + 'de la Muralla China. ¡Le faltan ' +  f"{math.trunc(21000-km):,}".replace(',','.') + ' kilómetros para recorrerla completa!'
    elif km > 21000 and km <= 40075: #vuelta al mundo
        porcentaje = math.trunc((km*100)/40075)
        en_pasos = 40075*1312
        if 52 >  porcentaje >  47:
            frase = 'Tu organización ha recorrido la mitad de una vuelta al mundo. ¡Le faltan ' + f"{math.trunc(40075-km):,}".replace(',','.') + ' kilómetros para dar una vuelta completa!'
        elif 100 >=  porcentaje >=  97:
            frase = '¡Felicidades! Tu organización ha dado una vuelta completa al mundo. ¡Esos son 40.075 kilómetros!'
        else:
            frase =  'Tu organización ha recorrido el ' + str(porcentaje) + '% ' + 'de una vuelta al mundo. ¡Le faltan ' +  f"{math.trunc(40075-km):,}".replace(',','.') + ' kilómetros para dar una vuelta completa!'
    elif km > 40075 and km < 384400:
        
        porcentaje = math.trunc((km*100)/384400)
        en_pasos = 40075*1312
        if 52 >  porcentaje >  47:
            frase = 'Tu organización ha recorrido la mitad del camino entre la Tierra y la Luna. ¡Le faltan ' + f"{math.trunc(384400-km):,}".replace(',','.') + ' kilómetros para llegar!'
        elif 100 >=  porcentaje >=  97:
            frase = '¡Felicidades! Tu organización ha recorrido todo el camino desde la Tierra y la Luna. ¡Esos son 384.400 kilómetros!'
        else:
            frase =  'Tu organización ha recorrido el ' + str(porcentaje) + '% ' + 'del camino entre la Tierra y la Luna. ¡Le faltan ' + f"{math.trunc(384400-km):,}".replace(',','.') + ' kilómetros para llegar!'
    elif km > 384400:
        porcentaje = math.trunc((km*100)/1400000)
        en_pasos = 40075*1312
        if 52 >  porcentaje >  47:
            frase = 'Tu organización ha recorrido la mitad de una vuelta al Sol. ¡Le faltan ' + f"{math.trunc(1400000-km):,}".replace(',','.') + ' kilómetros para completarla!'
        elif porcentaje > 100   or 100 >=  porcentaje >=  98:
            frase = '¡Felicidades! Tu organización ha dado una vuelta completa al Sol. ¡Esos son 1.400.000 kilómetros!'
        else:
            frase = 'Tu organización ha recorrido ' + str(porcentaje) + '% ' +  'de una vuelta al Sol.' '¡Le faltan ' + f"{math.trunc(1400000-km):,}".replace(',','.') + ' kilómetros para dar una vuelta completa!'


    return frase


In [29]:
def get_message_steps_month(steps: int):
    """
    Defines a message related to the distance associated to the total amount of steps on the organization.
    """ #cambiar a 1312 pasos por kilometro
    km = steps/1000
    if km < 11:
        viajes = math.trunc(km)
        if viajes > 1:
            frase = 'Tu organización subió ' + str(viajes) + ' veces el edifcio Burj Khalifa ¡El edifico más alto del mundo!'
        else:
            frase = 'Tu organización subió ' + str(viajes) + ' vez el edifcio Burj Khalifa ¡El edifico más alto del mundo!'
    elif 11 <= km <42:
        viajes = math.trunc(km/11)
        if viajes > 1:
            frase = 'Tu organización llegó ' + str(viajes) + ' veces hasta el fondo del mar'
        else:
            frase = 'Tu organización llegó ' + str(viajes) + ' vez hasta el fondo del mar'
    elif 42 <= km <= 3000:
        
        viajes = math.trunc(km/42)
        if viajes > 1:
            frase = 'Tu organización completó ' + str(viajes) + ' maratones este mes'
        else:
            frase = 'Tu organización completó ' + str(viajes) + ' maratón este mes'
    elif  km > 3000 and km <= 10000:
        viajes = math.trunc(km/1406)
        if viajes > 1:
            frase = 'Tu organización recorrió ' + str(viajes) + ' veces el trayecto de Santiago a Buenos Aires este mes'
        else:
            frase = 'Tu organización recorrió ' + str(viajes) + ' vez el trayecto de Santiago a Buenos Aires este mes'
            
    elif km > 10000 and km < 19640:
        
        viajes = math.trunc(km/7062)
        if viajes > 1:
            frase = 'Tu organización recorrió ' + str(viajes) + ' veces el Amazonas este mes'
        else:
            frase = 'Tu organización recorrió ' + str(viajes) + ' vez el Amazonas este mes'
    elif km >= 19640:
        
        viajes = math.trunc(km/19640)
        if viajes > 1:
            frase = 'Tu organización recorrió ' + str(viajes) + ' veces el trayecto de Chile a China este mes'
        else:
            frase = 'Tu organización recorrió ' + str(viajes) + ' vez el trayecto de Chile a China este mes'
    return frase

In [30]:
def get_included_benefits(df_benefits,organization):
    """
    Creates dictionary of included benefits on the organitzation's agreements
    """
    ben = df_benefits[df_benefits['name_organization']==organization]
    beneficios = set(ben['pk_dim_service'])
    dataframe = pd.DataFrame(["Seguro de vida por fallecimiento natural","Seguro de vida adicional fallecimiento accidental","Seguro de invalidez accidental","Atención médica en línea 24/7","Consulta psicológica en línea","Consulta nutricionista en línea","Asesoría de ahorro e inversión","Educación de bienestar", "Entrenamiento en casa","Meditación y bienestar"],columns=['nameBenefit'])
    dataframe['idBenefit']=[1,2,3,4,5,6,7,8,9,10]
    benefits1 = [8,7,6,1,2,9,10,15,4,5]
    benefits2 = [8,7,6,11,12,13,10,15,4,5]
    dataframe['benefits1'] = benefits1
    dataframe['benefits2'] = benefits2
    isIncluded=[]
    for i in range(0,10):
        if  dataframe.iloc[i,2] in beneficios:
            isIncluded.append(True)
        elif dataframe.iloc[i,3] in beneficios:
            isIncluded.append(True)
        else:
            isIncluded.append(False)
    dataframe['isIncluded'] = isIncluded
    dataframe = dataframe.drop(['benefits1','benefits2'],axis=1)
    ben_table = dataframe
    b_t = ben_table.to_dict('r')
    # return ben_table, b_t
    # print(b_t)
    return b_t

In [31]:
def dict_benefits(df_benefits,organization):
    """
    Creates a dictionary with the benefits of an organization and their percentage of activation. (third section of the report)
    """
    b_t = get_included_benefits(df_benefits,organization)
    dBenefits = {

      "included": b_t
    #   "activation": activation,
    }
    return  dBenefits

In [32]:
def get_top_5_sports(df_activities_by_day,organization):
    """
    Identify the top 5 sports by minutes on an organization
    """
    org = df_activities_by_day[df_activities_by_day['organization']==organization]
    org = org.groupby(['sport','id'])[['calories', 'minutes_activities']].sum().drop(['Sin Datos','Otros'], errors='ignore').reset_index()
    org = org.sort_values(by='minutes_activities',ascending=False).reset_index(drop = True)
    top5_sports = org[0:5].drop(columns = 'calories').rename(columns = {"sport": "nameActivity"})
    top5_sports['totalHours'] = np.trunc(top5_sports['minutes_activities']/60).astype(int)
    top5_sports = top5_sports.drop(columns = "minutes_activities")
    top5_sports = top5_sports.rename(columns={'id':'idActivity'})
    dict_sports = top5_sports.to_dict('r')
    return dict_sports


def get_meditation_perc(df_meditation,df_onboarding,organization):
    """
    Returns the percentage of the onboarded subscribers that meditate
    """
    onb = df_onboarding[df_onboarding['organization'] == organization]
    subs = onb['num_onboarding'].values[0]
    meditation=len(df_meditation[df_meditation['organization']==organization])
    meditation_perc = np.nan_to_num(100*meditation / subs)
    return meditation_perc

In [33]:
def dict_activities(df_steps, df_activities_by_day, df_meditation, df_onboarding,organization):
    """
    Creates a dictionary with the main activities of an organization and their performance compared to their previous report (fourth section of the report)
    """
    steps = df_steps[df_steps['organization']==organization]['steps'].sum()
    sum_minutes = df_meditation.loc[df_meditation['organization'] == organization, 'min_meditation'].sum()
    sum_minutes = 0 if sum_minutes is np.nan else math.trunc(sum_minutes)
    dActivities={
        "numSteps": math.trunc(steps/1500) ,
        "distanceMessage": get_message_steps(steps) ,
        "numActivities": len(df_activities_by_day[df_activities_by_day['organization']==organization]),
        "meditationPerc": round(get_meditation_perc(df_meditation, df_onboarding, organization),2),
        "meditationMin": sum_minutes,
    }
    return dActivities

In [34]:
def dict_meditation(df_meditation,df_onboarding,organization):
    """
    Creates a dictionary with the meditation of an organization (deprecated)
    """
    pctg_meditation = math.trunc(get_meditation_perc(df_meditation,df_onboarding,organization))
    mean_minutes = df_meditation.loc[df_meditation['organization'] == organization, 'min_meditation'].mean()
    mean_minutes = 0 if mean_minutes is np.nan else math.trunc(mean_minutes)
    d = pd.DataFrame([pctg_meditation,mean_minutes],['useMindfulness','meanMinutes']).transpose()
    dMeditation = d.to_dict('r')[0]
    return dMeditation

In [35]:
get_meditation_perc(df_meditation,df_onboarding,'NATURA')

0.5449591280653951

In [36]:
dict_meditation(df_meditation,df_onboarding,'NATURA')

C:\Users\ignac\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


{'useMindfulness': 0, 'meanMinutes': 1}

In [37]:
def bettercoins_360(df_onboarding,df_bettercoins_habits,df_cohorts,organization = None):
    """
    Bettercoins obtained by an organization per week per user only considering habits
    """
    df_total_days = df_cohorts.groupby('organization')[['total_days']].sum().fillna(0).reset_index()
    df_complete = df_onboarding.merge(df_bettercoins_habits,on='organization',how='outer')
    df_complete = df_total_days.merge(df_complete,on='organization',how='outer')
    df_complete.loc[:,'daily_bettercoins_subs'] = df_complete['bettercoins']/df_complete['total_days']
    df_complete =df_complete.replace([np.inf, -np.inf], np.nan).fillna(0)
    df_complete.to_excel('complete.xlsx', index=False)
    if organization is None:
        mean_cov = df_complete['daily_bettercoins_subs'].replace(0,np.nan).mean(skipna=True)
    else:
        mean_cov = df_complete[df_complete['organization']==organization]['daily_bettercoins_subs'].to_numpy()[0]
        if math.isnan(mean_cov):
            mean_cov=0
    return mean_cov*7

def minutes_360(df_onboarding,df_activities,df_cohorts,organization = None):
    """
    Minutes of exercises by an organization per week per user
    """
    df_total_days = df_cohorts.groupby('organization')[['total_days']].sum().fillna(0).reset_index()
    df_minutes = df_activities[['organization','numMinutes']].groupby(['organization'],as_index=False).sum()
    df_complete = df_onboarding.merge(df_minutes,on='organization',how='outer')
    df_complete = df_total_days.merge(df_complete,on='organization',how='outer')
    df_complete.loc[:,'weekly_minutes_subs'] = 7 * df_complete['numMinutes']/ df_complete['total_days']
    df_complete = df_complete.replace([np.inf, -np.inf], np.nan)
    df_complete = df_complete.fillna(0)
    if organization is None:
        mean_min = df_complete.mean(axis = 0,skipna = True)['weekly_minutes_subs']
    else:
        mean_min = df_complete[df_complete['organization']==organization]['weekly_minutes_subs'].to_numpy()[0]
    return mean_min

def meditation_360(df_onboarding,df_meditation,df_cohorts,organization = None):
    """
    Minutes of meditation by an organization per week per user
    """
    df_total_days = df_cohorts.groupby('organization')[['total_days']].sum().fillna(0).reset_index()
    df_minutes = df_meditation[['organization','min_meditation']].groupby(['organization'],as_index=False).sum()
    df_complete = df_onboarding.merge(df_minutes,on='organization',how='outer')
    df_complete = df_total_days.merge(df_complete,on='organization',how='outer')
    df_complete.loc[:,'weekly_minutes_subs'] = 7 * df_complete['min_meditation'] / df_complete['total_days']
    df_complete =df_complete.replace([np.inf, -np.inf], np.nan).fillna(0)
    if organization is None:
        mean_min = df_complete['weekly_minutes_subs'].mean(skipna=True)
    else:
        mean_min = df_complete[df_complete['organization']==organization]['weekly_minutes_subs'].to_numpy()[0]
        if math.isnan(mean_min):
            mean_min=0
    return mean_min


def donations_360(df_onboarding,df_donations,df_cohorts,organization = None):
    """
    Number of donations by an organization per week per user
    """
    df_total_days = df_cohorts.groupby('organization')[['total_days']].sum().fillna(0).reset_index()
    df_don = df_donations[['organization', 'donations']].groupby(['organization'],as_index=False).sum()
    df_complete = df_onboarding.merge(df_don,on='organization',how='outer')
    df_complete = df_total_days.merge(df_complete,on='organization',how='outer')
    df_complete.loc[:,'weekly_donations_subs'] = 7* df_complete['donations'] / df_complete['total_days']
    df_complete =df_complete.replace([np.inf, -np.inf], np.nan)
    df_complete.to_excel('complete2.xlsx', index=False)
    if organization is None:
        mean_donations = df_complete['weekly_donations_subs'].mean(skipna=True)
    else:
        mean_donations = df_complete[df_complete['organization']==organization]['weekly_donations_subs'].to_numpy()[0]
        if math.isnan(mean_donations):
            mean_donations=0
    return mean_donations

def mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation, organization = None):
    """
    Function that get you the mean of all organization per week per user for metrics and the values for a single organization
    """
    if organization is None:
        onboarding=(np.mean(df_onboarding['num_onboarding'].to_numpy()))
        onboarding_perc=(np.mean(df_onboarding['perc_activation'].to_numpy()))
    else:
        onboarding = (df_onboarding[df_onboarding['organization']==organization]['num_onboarding'].to_numpy()[0])
        onboarding_perc = (df_onboarding[df_onboarding['organization']==organization]['perc_activation'].to_numpy()[0])
    bettercoins=(np.nan_to_num(bettercoins_360(df_onboarding,df_bettercoins_habits,df_cohorts,organization)))
    minutes=(np.nan_to_num(minutes_360(df_onboarding,df_activities,df_cohorts,organization)))
    meditation=(np.nan_to_num(meditation_360(df_onboarding,df_meditation,df_cohorts,organization)))
    food=(np.nan_to_num(donations_360(df_onboarding,df_donations,df_cohorts,organization)))
    return [int(onboarding),int(minutes),int(bettercoins),int(meditation),int(food)],[int(onboarding_perc),int(np.min([minutes*100/180,100])),int(np.min([bettercoins*100/336,100])),int(np.min([meditation*100/10,100])),int(np.min([food*100,100]))]

In [38]:



def last_report(dir_reports):
    """
    Creates a dataframe with the performance of an organization in the previous report
    """
    dir_files = os.listdir(dir_reports)
    matrix_info = []
    for report in dir_files:
        f= open(os.path.join(dir_reports,report))
        data = json.load(f)
        org = data['organization']
        steps = data['activities']['numSteps']
        meditation = data['activities']['meditationMin']
        info = [org, steps, meditation]
        matrix_info.append(info)
        f.close()
    df_last_Q = pd.DataFrame(matrix_info,columns=['organization','steps', 'meditation'])
    return df_last_Q

def dict_last_report(df_steps, df_meditation, df_last_Q, organization):
    """
    Creates a dictionary with the info from the previous report if had otherwise 0 values are sent
    """
    not_sent = df_last_Q[df_last_Q['organization']==organization].empty
    if not not_sent:
        sent = True
        steps = df_steps[df_steps['organization']==organization]['steps'].sum()
        last_steps = df_last_Q[df_last_Q['organization']==organization]['steps'].values[0]
        last_meditation = df_last_Q[df_last_Q['organization']==organization]['meditation'].values[0]
        sum_minutes = df_meditation.loc[df_meditation['organization'] == organization, 'min_meditation'].sum()
        sum_minutes = 0 if sum_minutes is np.nan else math.trunc(sum_minutes)
        per_steps = round(100*(steps-last_steps)/last_steps, 2)
        per_meditation = round(100*(sum_minutes-last_meditation)/last_meditation,2) if last_meditation !=0 else "No aplica"
    else:
        sent = False
        per_steps = 0
        per_meditation = 0
    dLastReport={
        "sentLastQ": sent,
        "perNumSteps": per_steps ,
        "perMeditationMin": per_meditation
    }
    return dLastReport

def dict_levels(df_levels,organization):
    """
    Returns dictionary with the biggest level who have at least 5% of the users
    """
    level = df_levels[df_levels['name_organization'] == organization]
    tot = int(level['athlete'].sum())
    level['perc'] = round(level['athlete']/tot*100)
    level = level.drop(columns = "name_organization")
    level['nivel'] = pd.Categorical(level['nivel'],categories=[1,2,3,4,5],ordered=True)
    level = level.sort_values('nivel',ascending=True)
    level = level.replace({'nivel': {1: 'Rookie', 2: 'Giver',3: 'SuperGiver',4: 'UltimateGiver',5: 'Betterflyer'}})
    aux = True
    i = 1
    while aux:
        check_level = level.iloc[-i]
        if check_level['perc'].astype(int) >= 5:
            aux = False
        i += 1
    return check_level

In [39]:
def dict_impact(df_donations,organization):
    c = df_donations[df_donations['organization']==organization]
        # if organization=='SANTANDER':
    #     df =  df_donations[(df_donations['organization']==organization) & ((df_donations['id_campaign']!=17) ) & ((df_donations['id_campaign']!=18) ) & ((df_donations['id_campaign']!=21) )]
    # else:
    #     df = df_donations
    # donations = list(df[df['organization']==organization]['donations'])
    # campaign = list(df[df['organization']==organization]['campaign'])
    # id_campaign = list(df[df['organization']==organization]['id_campaign'])
    # type_donation = list(df[df['organization']==organization]['type_donation'])
    # todo = list(zip(donations,campaign,id_campaign,type_donation))
    if organization == "SANTANDER":
        agua = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        arboles = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        comida = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        


        if 33 in c['id_campaign'].tolist():
            ukraine = {
                'quantity': int(c[c['id_campaign']==33]['donations'].sum()),
                'cause': c[c['id_campaign']==33]['campaign'].to_list(),
                'idCause': c[c['id_campaign']==33]['id_campaign'].to_list(),
                'hasDonation': bool(c[c['id_campaign']==33]['donations'].sum() > 0)
            }
        else:
            ukraine = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
            }
        generic = {
            'quantity': int(c[c['type_donation']=='Generic']['donations'].sum()),
            'cause': c[c['type_donation']=='Generic']['campaign'].to_list(),
            'idCause': c[c['type_donation']=='Generic']['id_campaign'].to_list(),
            'hasDonation': bool(c[c['type_donation']=='Generic']['donations'].sum() > 0)
        }    
    elif organization == "CRUZADOS":
        agua = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        arboles = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        comida = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
        }
        if 33 in c['id_campaign'].tolist():
            ukraine = {
                'quantity': int(c[c['id_campaign']==33]['donations'].sum()),
                'cause': c[c['id_campaign']==33]['campaign'].to_list(),
                'idCause': c[c['id_campaign']==33]['id_campaign'].to_list(),
                'hasDonation': bool(c[c['id_campaign']==33]['donations'].sum() > 0)
            }
        else:
            ukraine = {
                'quantity': 0,
                'cause': [],
                'idCause': [],
                'hasDonation': False
            }
        generic = {
            'quantity': int(c[c['type_donation']=='Meals']['donations'].sum()),
            'cause': c[c['type_donation']=='Meals']['campaign'].to_list(),
            'idCause': c[c['type_donation']=='Meals']['id_campaign'].to_list(),
            'hasDonation': bool(c[c['type_donation']=='Meals']['donations'].sum() > 0)
        }  
    else:
        agua = {
            'quantity': int(c[c['type_donation']=='Water']['donations'].sum()),
            'cause': c[c['type_donation']=='Water']['campaign'].to_list(),
            'idCause': c[c['type_donation']=='Water']['id_campaign'].to_list(),
            'hasDonation': bool(c[c['type_donation']=='Water']['donations'].sum() > 0)
        }
        arboles = {
            'quantity': int(c[c['type_donation']=='Trees']['donations'].sum()),
            'cause': c[c['type_donation']=='Trees']['campaign'].to_list(),
            'idCause': c[c['type_donation']=='Trees']['id_campaign'].to_list(),
            'hasDonation': bool(c[c['type_donation']=='Trees']['donations'].sum() > 0)
        }
        comida = {
            'quantity': int(c[c['type_donation']=='Meals']['donations'].sum()),
            'cause': c[c['type_donation']=='Meals']['campaign'].to_list(),
            'idCause': c[c['type_donation']=='Meals']['id_campaign'].to_list(),
            'hasDonation': bool(c[c['type_donation']=='Meals']['donations'].sum() > 0)
        }
        


        if 33 in c['id_campaign'].tolist():
            ukraine = {
                'quantity': int(c[c['id_campaign']==33]['donations'].sum()),
                'cause': c[c['id_campaign']==33]['campaign'].to_list(),
                'idCause': c[c['id_campaign']==33]['id_campaign'].to_list(),
                'hasDonation': bool(c[c['id_campaign']==33]['donations'].sum() > 0)
            }


        


        generic = {
            'quantity': int(c[(c['type_donation']=='Generic') & (c['id_campaign']!=33) ]['donations'].sum()),
            'cause': c[(c['type_donation']=='Generic') & (c['id_campaign']!=33)]['campaign'].to_list(),
            'idCause': c[(c['type_donation']=='Generic') & (c['id_campaign']!=33)]['id_campaign'].to_list(),
            'hasDonation': bool(c[(c['type_donation']=='Generic') & (c['id_campaign']!=33)]['donations'].sum() > 0)
        }
            

    dImpact = {
    "Agua": agua,
    "Arboles": arboles,
    "Comida": comida,
    "Ucrania":ukraine,
    "Generic": generic
    }
    return dImpact
  
    
    

In [40]:
df_donations[df_donations['organization']=='Betterfly']['id_campaign'].tolist()

[13, 17, 18, 20, 21, 26, 29, 30, 33]

In [41]:
df_donations[df_donations['organization']=='SANTANDER']

,id_campaign,organization,campaign,type_donation,donations
533,13,SANTANDER,Feed a child in Venezuela,Meals,10
1244,17,SANTANDER,Fight the water crisis,Water,4
1971,18,SANTANDER,Trees to save the future,Trees,9
2093,19,SANTANDER,Santander supports education,Generic,13167
2646,20,SANTANDER,Against child malnutrition in Haiti,Meals,3
3343,21,SANTANDER,Chile's first food bank,Meals,3


In [42]:
dict_impact(df_donations,'SANTANDER')

{'Agua': {'quantity': 0, 'cause': [], 'idCause': [], 'hasDonation': False},
 'Arboles': {'quantity': 0, 'cause': [], 'idCause': [], 'hasDonation': False},
 'Comida': {'quantity': 0, 'cause': [], 'idCause': [], 'hasDonation': False},
 'Ucrania': {'quantity': 0, 'cause': [], 'idCause': [], 'hasDonation': False},
 'Generic': {'quantity': 13167,
  'cause': ['Santander supports education'],
  'idCause': [19],
  'hasDonation': True}}

In [43]:

def dict_cluster(df_cluster, organization):
    """
    Returns the percentage of the onboarded subscribers that meditate
    """
    clust = df_cluster[df_cluster['name_organization'] == organization]
    clust = clust[(clust['labels'] != 'No physical activity') & (clust['labels'] != 'No information') & (clust['labels'] != 'No sufficient information')]
    tot = int(clust['freq'].sum())
    clust['perc'] = round(clust['freq']/tot*100)
    clust = clust.drop(columns = "name_organization")
    clust['labels'] = pd.Categorical(clust['labels'],categories=['Beginners','Walkers','Amateurs', 'Athletes'],ordered=True)
    clust = clust.sort_values('labels',ascending=True)
    clust = clust.replace({'labels': {'Beginners': 'Principiantes', 'Walkers': 'Caminantes','Amateurs': 'Amateurs', 'Athletes': 'Atletas'}})
    dClust = {
        'total': tot,
        'clusters': clust.to_dict('r')
    }
    return dClust

In [44]:

def datos_mejor_mes(df_steps,df_donations_b,df_meditation_mes,df_activities_by_day,organization):
    steps_year = df_steps[df_steps['organization']==organization]['steps'].sum()
    donations_meal = df_donations_b[(df_donations_b['organization'] == organization)& (df_donations_b['type_donation']== 'Meals')]['donations'].sum()
    meditation = df_meditation_mes[df_meditation_mes['organization']==organization].groupby(['organization']).agg({'num_meditations':'sum','min_meditation':'sum'}).reset_index()['min_meditation'].values.tolist()[0]
    routine_actividad = df_activities_by_day[(df_activities_by_day['organization']== organization) ]['sport'].count()
    if organization == 'CRUZADOS' or organization=='SANTANDER':
        donations_meal = 0
    dActivities={
        "numSteps":math.trunc(steps_year/1500),
        "message_year": get_message_steps(steps_year),
        "donations": donations_meal,
        "min_meditation":meditation,
        "routine_activity":routine_actividad}

    # print(total_donations)
    return dActivities
    

In [45]:
allOrganizations =  mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation)[1]

In [46]:
datos_mejor_mes(df_steps,df_donations_b,df_meditation_mes,df_activities_by_day,'KUEHNE+NAGEL')

{'numSteps': 28896,
 'message_year': 'Tu organización ha recorrido el 72% de una vuelta al mundo. ¡Le faltan 11.178 kilómetros para dar una vuelta completa!',
 'donations': 0,
 'min_meditation': 2619.0,
 'routine_activity': 2515}

In [47]:
orgs = df_onboarding['organization'].tolist()[20:30]

In [49]:

allOrganizations =  mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation)[1]
errores=0
for organization in orgs:
    try:
        #print("Generating report for: " + organization)
            # Summary section
        dSummary = dict_summary(df_onboarding, df_activities, df_coverage, df_donations, df_bettercoins, df_levels, organization)
            # Level section
        #dLevel = dict_levels(df_levels, organization)
            # Impact section
        dImpact =  dict_impact(df_donations,organization)

            # Benefits section
        dBenefits = dict_benefits(df_benefits,organization)

            # Activities section
        dActivities = datos_mejor_mes(df_steps_mes,df_donations_b,df_meditation_mes,df_activities_by_day,organization)

        vision360 = mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation, organization)

            #idOrganization
        idOrganization= int(df_onboarding[df_onboarding['organization']==organization]['id'].values[0])
        # bettertogether
        dcluster = dict_cluster(df_cluster, organization)
        if organization in df_manual['name_organization'].tolist():
            manual_act = int(df_manual[df_manual['name_organization']==organization]['atleta'].values[0])   
        else:
            manual_act = 0


        dict_final={
                "organization": organization,
                "idOrganization": idOrganization ,
                "date": report_date.strftime("%d/%m/%Y"),
                "period": "Q4 2021",
                "summary":dSummary,
                "vision360": {
                    "metrics": ["Porcentaje de enrolamiento","Minutos de actividad física","Bettercoins ganadas", "Minutos de meditación", "Cantidad de donaciones"],
                    "units": ["%", "minutos semanales por colaborador", "bettercoins semanales por colaborador", "minutos semanales por colaborador", "donaciones semanales por colaborador"],
                    "achievedValue":vision360[0],
                    "sugestedValue": [dSummary['numSuscribers'],180,420,10,1] ,
                    "achievedPercentage": vision360[1] ,
                    "allOrganizations": allOrganizations
                },
                "impact":dImpact,
                "benefits":dBenefits,
                "activities": dActivities,
                'manual_habits':manual_act,
                'bettertogether': dcluster
                

        }

        if dSummary['numDonations'] > 0:
                json_object = json.dumps(dict_final, indent = 2,default=str,ensure_ascii=False)
                with open("outputq1/Q1_" + str(today_str)+'_'+ str(idOrganization) +".json", "w", encoding="utf8") as outfile:
                    outfile.write(json_object)
    except:
        print(organization)
        
        errores +=1
print(errores)




C:\Users\ignac\AppData\Local\Temp/ipykernel_18408/651662895.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level['perc'] = round(level['athlete']/tot*100)
C:\Users\ignac\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


CARIOLA


C:\Users\ignac\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
C:\Users\ignac\AppData\Local\Temp/ipykernel_18408/651662895.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level['perc'] = round(level['athlete']/tot*100)
C:\Users\ignac\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Us

1


In [47]:

organization='CGE'
allOrganizations =  mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation)[1]

print("Generating report for: " + organization)
    # Summary section
dSummary = dict_summary(df_onboarding, df_activities, df_coverage, df_donations, df_bettercoins, df_levels, organization)
    # Level section
#dLevel = dict_levels(df_levels, organization)
    # Impact section
dImpact =  dict_impact(df_donations,organization)

    # Benefits section
dBenefits = dict_benefits(df_benefits,organization)

    # Activities section
dActivities = datos_mejor_mes(df_steps_mes,df_donations_b,df_meditation_mes,df_activities_by_day,organization)

vision360 = mean_organization(df_onboarding, df_cohorts, df_bettercoins_habits, df_activities, df_donations, df_meditation, organization)

    #idOrganization
idOrganization= int(df_onboarding[df_onboarding['organization']==organization]['id'].values[0])

if organization in df_manual['name_organization'].tolist():
    manual_act = int(df_manual[df_manual['name_organization']==organization]['atleta'].values[0])   
else:
    manual_act = 0
# bettertogether
dcluster = dict_cluster(df_cluster, organization)

dict_final={
        "organization": organization,
        "idOrganization": idOrganization ,
        "date": report_date.strftime("%d/%m/%Y"),
        "period": "Q4 2021",
        "summary":dSummary,
        "vision360": {
            "metrics": ["Porcentaje de enrolamiento","Minutos de actividad física","Bettercoins ganadas", "Minutos de meditación", "Cantidad de donaciones"],
            "units": ["%", "minutos semanales por colaborador", "bettercoins semanales por colaborador", "minutos semanales por colaborador", "donaciones semanales por colaborador"],
            "achievedValue":vision360[0],
            "sugestedValue": [dSummary['numSuscribers'],180,420,10,1] ,
            "achievedPercentage": vision360[1] ,
            "allOrganizations": allOrganizations
        },
        "impact":dImpact,
        "benefits":dBenefits,
        "activities": dActivities,
        'manual_habits':manual_act,
        'bettertogether': dcluster
        

}


json_object = json.dumps(dict_final, indent = 2,default=str,ensure_ascii=False)
with open("Q1_" + str(today_str)+'_'+ str(idOrganization) +".json", "w", encoding="utf8") as outfile:
    outfile.write(json_object)

Generating report for: CGE


C:\Users\ignac\AppData\Local\Temp/ipykernel_13028/651662895.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  level['perc'] = round(level['athlete']/tot*100)
C:\Users\ignac\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [62]:
df_levels

,name_organization,nivel,athlete
0,23People,1,44
1,3G OFFICE,1,1
2,3G OFFICE,2,3
3,3G OFFICE,3,3
4,4Geeks CL SpA,1,5
...,...,...,...
1753,YOUTOPIA,4,2
1754,fiiS,1,1
1755,uPlanner,1,26
1756,uPlanner,2,12


In [65]:
df_levels[df_levels['name_organization']=='CARIOLA']

,name_organization,nivel,athlete


In [60]:
dict_levels(df_levels, organization)

IndexError: single positional indexer is out-of-bounds

In [59]:

dict_cluster(df_cluster,'LA COFRADÍA')

{'total': 0, 'clusters': []}